# Data exploration and Transformation

*Note If use this notebook in Google Colab please upload all the data and indicate the path.*

This notebook is dedicated to exploratory analyses of the ATIS dataset.
Some key observations:
1. Highly imbalanced dataset!
2. Time in the dataset has different formats.
3. Looks like the text is lowercased  -> need to take into account this at inference time in prod to avoid data shift.
4. No duplicates in the datasets, however, there is data leakage between train and test sets.
5. There is some label mismatch in train and test sets.
6. Some labels are represented as a combination of several other intents (labels) from the dataset.
7. Looks like punctuation was removed -> need to take into account this at inference time in prod.
8. The text length is relatively short -> we may think about reducing the sequence length in Transformer for faster inference time (future work).
9. Some labels are not clean.
10. Some classes are duplicated, e.g., "flight+airfare" and "airfare+flight" in the test set, "flight_no+airline" to "airline+flight_no" in both sets.

--> The dataset needs cleaning and transformation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Load train and test datasets into DataFrames
df = pd.read_csv("data/atis/train.tsv", sep="\t", names=["text", "label"])
df_test = pd.read_csv("data/atis/test.tsv", sep="\t", names=["text", "label"])

# Remove white spaces from corners
df["text"] = df["text"].apply(lambda x: x.strip())
df_test["text"] = df_test["text"].apply(lambda x: x.strip())

print(f"len df: {len(df)}")
print(f"len df_test: {len(df_test)}")

df.head(2)

len df: 4634
len df_test: 850


,text,label
0,i want to fly from boston at 838 am and arrive...,flight
1,what flights are available from pittsburgh to ...,flight


# Sanity checks for duplicates and data leakage in the datasets

In [ ]:
import hashlib

def get_sha(text: str) -> bytes:
    sha256 = hashlib.sha256()
    sha256.update(str.encode(text))
    return sha256.digest()

# Compute hash for text column
df["text_hash"] = df["text"].map(get_sha)
df_test["text_hash"] = df_test["text"].map(get_sha)

# Check train dataframe
print("* Check TRAIN dataframe:* ")
print(f"Data contains duplicates? -> {len(df) != len(df.text_hash.drop_duplicates())}")
print(f"Data contains duplicates in 'text' field? -> {len(df) != len(df.text_hash.drop_duplicates())}")
print("----------------")
# Check test dataframe
print("* Check TEST dataframe:* ")
print(f"Data contains duplicates? -> {len(df_test) != len(df_test.drop_duplicates())}")
print(f"Data contains duplicates in 'text' field? -> {len(df_test) != len(df_test.text_hash.drop_duplicates())}")
print("----------------")
# Check Train vs Test data
samples_incommon_hash = set(df["text_hash"].values) & set(df_test["text_hash"].values)
num_samples_incommon = len(samples_incommon_hash)
print(f"* Number of samples in common in train and test sets: {num_samples_incommon} *")

* Check TRAIN dataframe:* 
Data contains duplicates? -> False
Data contains duplicates in 'text' field? -> False
----------------
* Check TEST dataframe:* 
Data contains duplicates? -> False
Data contains duplicates in 'text' field? -> False
----------------
* Number of samples in common in train and test sets: 11 *


In [ ]:
# Understand if shared {text, label} are consistent in values in both sets
df_common = df[df["text_hash"].isin(samples_incommon_hash)]
df_test_common = df_test[df_test["text_hash"].isin(samples_incommon_hash)]
print("Text-lable consistency preserved: ", end="")
print(set(df_common["text"] + df_common["label"]) == set(df_test_common["text"] + df_test_common["label"]))

# Print common samples
shared_samples_indx_train = df_common[["text", "label"]].index
print(shared_samples_indx_train)
df_common[["text", "label"]]

Text-lable consistency preserved: True
Int64Index([254, 729, 827, 1375, 1543, 1846, 2220, 2722, 2962, 3009, 4371], dtype='int64')


,text,label
254,give me the flights from pittsburgh to los ang...,flight
729,what is fare code m,abbreviation
827,what does fare code qo mean,abbreviation
1375,give me a flight from charlotte to baltimore o...,flight
1543,what does fare code y mean,abbreviation
1846,what airline is as,airline
2220,what airline is hp,airline
2722,what airline is dl,airline
2962,ground transportation in las vegas,ground_service
3009,what is fare code f,abbreviation


_--> The labels above are among major classes, thus we can safely remove them from the train set and keep them in the test set._

In [ ]:
print(f"Size Train set: {len(df)}")
df.drop(index=shared_samples_indx_train, inplace=True)
print(f"Size Train set new: {len(df)}")

Size Train set: 4634
Size Train set new: 4623


In [ ]:
# Drop hash column
df.drop(columns=["text_hash"], inplace=True)
df_test.drop(columns=["text_hash"], inplace=True)

# Look into the label distribution

In [ ]:
print(f"Label Distr. in Train dataset:\n{df.label.value_counts()}")
# Similar imbalanced distribution is in test set as well

Label Distr. in Train dataset:
flight                        3424
airfare                        403
ground_service                 233
airline                        145
abbreviation                   104
aircraft                        78
flight_time                     52
quantity                        49
distance                        20
city                            18
airport                         18
ground_fare                     17
flight+airfare                  17
capacity                        16
flight_no                       12
meal                             6
restriction                      5
airline+flight_no                2
ground_service+ground_fare       1
airfare+flight_time              1
cheapest                         1
aircraft+flight+flight_no        1
Name: label, dtype: int64


In [ ]:
print(f"Label Distr. in Test dataset:\n{df_test.label.value_counts()}")

Label Distr. in Test dataset:
flight               613
airfare               48
ground_service        36
airline               28
abbreviation          26
capacity              21
airport               13
flight+airfare        12
distance              10
aircraft               8
flight_no              8
ground_fare            7
meal                   6
city                   5
quantity               3
day_name               2
flight_time            1
airfare+flight         1
flight+airline         1
flight_no+airline      1
Name: label, dtype: int64


_-->Looks like "airfare+flight" and "flight+airfare" in test are the same classes --> fix it!_

In [ ]:
# Get samples with "duplicated" classes in test set
print(df_test[df_test["label"] == "airfare+flight"])
_idx1 = df_test[df_test["label"] == "airfare+flight"].index

# Change the label name from "airfare+flight" to "flight+airfare"
df_test.loc[_idx1, "label"] = "flight+airfare"

                                                  text           label
228  list the airfare for american airlines flight ...  airfare+flight


In [ ]:
# Get samples with "duplicated" classes in test set
print(df_test[df_test["label"] == "flight_no+airline"])
_idx2 = df_test[df_test["label"] == "flight_no+airline"].index

# Change the label name in test from "flight_no+airline" to "airline+flight_no"
df_test.loc[_idx2, "label"] = "airline+flight_no"

                                                  text              label
493  i need flight numbers and airlines for flights...  flight_no+airline


In [ ]:
# Check the label consistency in train and test sets
print(f"Do labels from Train set match labels in Test: {set(df.label.values) == set(df_test.label.values)} !")

labels_train = set(df.label.unique())
labels_test = set(df_test.label.unique())
all_labels_set = labels_train | labels_test
print(f"Total number of classes: {len(all_labels_set)}")

Do labels from Train set match labels in Test: False !
Total number of classes: 24


### Understand which classes are preserved in Test set but missed in the Train set and vice versa

In [ ]:
missed_labels_in_train = labels_test - labels_train
print(f"Missed labels in Train set: {missed_labels_in_train}")
# Visualise distribution of this classes in test set to see if we can move some of samples in train set
df_test[df_test["label"].isin(missed_labels_in_train)]

Missed labels in Train set: {'day_name', 'flight+airline'}


,text,label
34,what day of the week do flights from nashville...,day_name
36,what days of the week do flights from san jose...,day_name
486,i need flight and airline information for a fl...,flight+airline


_--> At this moment two classes are missed in the train set._

#### Looking at Test set now

In [ ]:
missed_labels_in_test = labels_train - labels_test
print(f"Missed labels in Test set: {missed_labels_in_test}")
# Visualise distribution of this classes in test set to see if we can move some of samples in train set
df[df["label"].isin(missed_labels_in_test)]

Missed labels in Test set: {'ground_service+ground_fare', 'airfare+flight_time', 'aircraft+flight+flight_no', 'cheapest', 'restriction'}


,text,label
830,what are the air restrictions on flights from ...,restriction
1167,what is restriction ap57,restriction
2596,what ground transportation is available from t...,ground_service+ground_fare
2700,show me the costs and times for flights from s...,airfare+flight_time
3268,show me the cheapest fare in the database,cheapest
3668,i want to fly from detroit to st. petersburg o...,aircraft+flight+flight_no
3885,what are the restrictions on the cheapest one ...,restriction
3985,what 's restriction ap68,restriction
4452,what is restriction ap80,restriction


_--> All of these samples are among low-represented classes in the train set --> we may consider to keep them in train set only._

--> At this point, I think about two possibilities for the task: 1) Join low-represented classes in a new class "Other", or 2) Manually (or outsource) create a view of more samples for these classes.  I prefer to go with the second approach as it is low effort in this case and we will save the original label set.

#### Generate more samples for low-represented classes

 _*Before manually creating new samples I will move some original samples from train to test dataset so that in the test set we cover all classes in this problem._

In [ ]:
_idx3 = [830, 1167, 2596, 2700, 3268, 3668]
print(f"Size Train set: {len(df)}")
print(f"Size Test set: {len(df_test)}")

df_test = pd.concat([
    df_test,
    df.loc[_idx3].copy(deep=True),
], ignore_index=True)
df.drop(index=_idx3, inplace=True)

print(f"Size Train NEW set: {len(df)}")
print(f"Size Test NEW set: {len(df_test)}")

Size Train set: 4623
Size Test set: 850
Size Train NEW set: 4617
Size Test NEW set: 856


In [ ]:
# Manually create data to have at least 5 samples per class
new_train_samples_list = [
    ("do you provide any meal for my flight", "meal"),
    ("what kind of meal do you give for flight 751", "meal"),
    ("what are vegan dishes available", "meal"),
    ("can i need to order food in advance", "meal"),
    ("what are the air restrictions on flights from chicago to new york", "restriction"),
    ("is there any luggage restrictions", "restriction"),
    ("can i bring a dog on board", "restriction"),
    ("any restrictions apply on the flight ad34", "restriction"),
    ("what are the restrictions on fare between los angeles and austin", "restriction"),
    ("what airline and flight i can buy from columbus to miami", "airline+flight_no"),
    ("i would like to know airlines that fly to san francisco and numbers of flights", "airline+flight_no"),
    ("is there any and airlines and flight number for today from chicago", "airline+flight_no"),
    ("what ground transportation is available in dallas tomorrow and what are the prices", "ground_service+ground_fare"),
    ("show ground transportation for pittsburgh together with prices", "ground_service+ground_fare"),
    ("give cheapest ground transportation in baltimore", "ground_service+ground_fare"),
    ("ground transfer in san francisco and prices", "ground_service+ground_fare"),
    ("economy fares new york to miami round trip and show me times for flights", "airfare+flight_time"),
    ("how much is a flight from washington to montreal provide list the flight times", "airfare+flight_time"),
    ("what are the costs of flights from dallas to boston tomorrow and time", "airfare+flight_time"),
    ("all fares and flights from philadelphia and display time", "airfare+flight_time"),
    ("show me the cheapest flight from dallas to san francesco", "cheapest"),
    ("round trip cheapest fares to philadelphia", "cheapest"),
    ("what is the lowest priced fare from baltimore to boston", "cheapest"),
    ("cheapest flight for today", "cheapest"),
    ("show me the type of aircraft that cp uses by this flight and tell me the flight number", "aircraft+flight+flight_no"),
    ("i would like a to know the type of aircraft flight number from atlanta to boston on friday used by this flight", "aircraft+flight+flight_no"),
    ("what flight number leaving denver flying to san francisco what type of aircraft is used for this flight", "aircraft+flight+flight_no"),
    ("what kind of airplane goes from boston to san francisco before noon and what flight and flight number", "aircraft+flight+flight_no"),
    ("i would like the day name for flight from washington to philadelphia", "day_name"),
    ("what is the day of the week of flights from boston to denver", "day_name"),
    ("what is the day of flights from pittsburgh to dallas", "day_name"),
    ("what day of the week does the flight leave denver going to san francisco", "day_name"),
    ("what is the day of week for flights between pittsburgh and boston on the evening in july", "day_name"),
    ("please show me round trip tickets from denver to oakland for american airlines", "flight+airfare"),
    ("what airlines and the price one way from atlanta to denver", "flight+airfare"),
    ("how much is a trip fare from memphis what kind of airline ", "flight+airfare"),
    ("what's the trip fare from dallas to atlanta and whichairlines", "flight+airfare"),
    ("what airlines have flights from baltimore to seattle and list the number of flights", "airline+flight_no"),
    ("which airlines fly from boston to washington dc what are the flight numbers", "airline+flight_no"),
    ("flight numbers and airline that stands for dl", "airline+flight_no"),
    ("flight numbers and airline that fly to seattle from pittsburgh", "airline+flight_no"),
    ("francisco to pittsburgh airlines and flight numbers leaving on monday night", "airline+flight_no"),
    ("show me all flights and airlines both direct from dallas to either san francisco or oakland", "flight+airline"),
    ("give me flights from chicago to seattle and operating arilines", "flight+airline"),
    ("tell me what airline and the earliest flight leaves between atlanta and denver", "flight+airline"),
    ("i want a flight from toronto to montreal that leaves early friday morning give airlines available", "flight+airline"),
    ("what first class flights and airlines are available from denver to baltimore on october fifth", "flight+airline"),
]

In [ ]:
# Add generated samples to train set
df = pd.concat([
    df,
    pd.DataFrame(new_train_samples_list, columns = ["text", "label"]),
], ignore_index=True)
print(f"Label Distr. NEW in Train dataset:\n{df.label.value_counts()}")

Label Distr. NEW in Train dataset:
flight                        3424
airfare                        403
ground_service                 233
airline                        145
abbreviation                   104
aircraft                        78
flight_time                     52
quantity                        49
flight+airfare                  21
distance                        20
airport                         18
city                            18
ground_fare                     17
capacity                        16
flight_no                       12
meal                            10
airline+flight_no               10
restriction                      8
day_name                         5
flight+airline                   5
ground_service+ground_fare       4
airfare+flight_time              4
cheapest                         4
aircraft+flight+flight_no        4
Name: label, dtype: int64


In [ ]:
#### Check again if we now have any missed labels in train-test sets

In [ ]:
labels_train = set(df.label.unique())
labels_test = set(df_test.label.unique())
all_labels_set = labels_train | labels_test
missed_labels_in_train = labels_test - labels_train
print(f"Missed labels in Train set: {missed_labels_in_train}")
missed_labels_in_test = labels_train - labels_test
print(f"Missed labels in Test set: {missed_labels_in_test}")

Missed labels in Train set: set()
Missed labels in Test set: set()


--> Nice, all the classes are presented in both sets!

### Add unique IDs for each data row
Can be usseful to log samples used for model training, e g, for a Model card.

In [ ]:
import uuid

# generate UUID
df["uuid"] = df["text"].apply(lambda _: uuid.uuid4())
df_test["uuid"] = df_test["text"].apply(lambda _: uuid.uuid4())

### Train-Validation data split

In [ ]:
from sklearn.model_selection import train_test_split

seed = 42

y = df.get("label")
X = df

# split dataset in train val sets
X_train, X_val, y_train,y_val = train_test_split(X.index, y, random_state=seed, test_size=0.2, stratify=y)

df_train = df.loc[X_train]
df_val = df.loc[X_val]

assert set(df.loc[X_val].label.values) == set(df.loc[X_train].label.values)

In [ ]:
# save train val test datasets
df_train.to_csv("data/atis/train_transformed.csv", index=False)
df_val.to_csv("data/atis/val_transformed.csv", index=False)
df_test.to_csv("data/atis/test_transformed.csv", index=False)